In [53]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os

### **1.0 Get article links**

In [58]:
links = [
"gaming",
"peranti-komputer",
"tips-tricks",
"ulasan-review",
"bijak-berinternet"
]

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [57]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "lxml")

    block = soup.find('div',attrs = {"class":"cm-content"})
    
    if block is None:
        return

    for link in block.find_all('h2', attrs = {"class":"cm-entry-title"}):

        href = link.find('a').get('href')

        hrefs.append(href)

In [75]:
max_worker = 10

for t in links:
    
    hrefs = []
    print(t)
    url = f'https://mat-gaming.com/{t}'
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]

    if pgno == []:
        max_page = 1
    else:
        max_page = max([int(a_.split('page/')[1].replace('/','')) for a_ in pgno])

    print(max_page)
    
    # Since there's not many pages/articles on the site, 
    # it shouldn't be too complex or computation intensive. If there is 
    # a need for a new page (only for the "gaming" section),
    # there doesn't seem to have a page 1. 
    for i in tqdm(range(max_page)):
        if max_page == 1:
            urls = [f'{url}/']
        else:
            urls = [f'{url}/'] + [f'{url}/page/{a}' for a in pgno]
    
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()
    
    hrefs = list(set(hrefs))
    with open(f'mat-gaming-{t}-link.json', 'a') as f:
        json.dump(hrefs, f)

gaming
https://mat-gaming.com/gaming
2


100%|██████████| 2/2 [00:04<00:00,  2.13s/it]


peranti-komputer
https://mat-gaming.com/peranti-komputer
1


100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


tips-tricks
https://mat-gaming.com/tips-tricks
1


100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


ulasan-review
https://mat-gaming.com/ulasan-review
1


100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


bijak-berinternet
https://mat-gaming.com/bijak-berinternet
1


100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


In [76]:
url = []

for t in links:
    print(t)
    with open(f'mat-gaming-{t}-link.json') as fopen:
        href = json.load(fopen)
    url.extend(href)

gaming
peranti-komputer
tips-tricks
ulasan-review
bijak-berinternet


In [78]:
len(url)

53

In [80]:
# get distinct urls only (remove duplicates)
hrefs = list(set(url))

In [88]:
with open(f'mat-gaming-link.json', 'a') as f:
        json.dump(hrefs, f)

In [84]:
len(hrefs)

49

In [82]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="cm-entry-title").text
        h = soup.find('div', class_="cm-post-content")
        p = [p.text for p in h.find_all("p")]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [86]:
max_workers = 5

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('mat-gaming.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

  0%|          | 0/49 [00:00<?, ?it/s]

100%|██████████| 49/49 [00:12<00:00,  3.89it/s]
